In [ ]:
import os
import glob
import pandas as pd

# --- 1. Paths & filenames ---
DATA_DIR = r'Data/'
WEATHER_FILES = [
    'dallas.json','houston.json','la.json','nyc.json',
    'san_diego.json','san_jose.json','san_antonio.json',
    'phoenix.json','philadelphia.json','seattle.json'
]
DEMAND_FILES = {
    'nyc': 'cleaned_subregion_data.csv',
    'phoenix': 'cleaned_balance_data.csv',
    'seattle': 'cleaned_balance_data.csv',
    'houston': 'cleaned_texas_data.csv',
    'san antonio': 'cleaned_texas_data.csv',
    'dallas': 'cleaned_texas_data.csv'
}

# --- 2. Load & unify weather JSONs ---
weather_dfs = []
for fname in WEATHER_FILES:
    city_key = os.path.splitext(fname)[0].replace('_', ' ').lower()
    file_path = os.path.join(DATA_DIR, fname)
    df = pd.read_json(file_path)
    df['city'] = city_key
    df['timestamp'] = pd.to_datetime(df['time'], unit='s')
    df.drop(columns=['time'], inplace=True)
    weather_dfs.append(df)
weather_df = pd.concat(weather_dfs, ignore_index=True)

# --- 3. Load & unify demand sources ---
demand_dfs = []
for city, fname in DEMAND_FILES.items():
    path = os.path.join(DATA_DIR, fname)
    df = pd.read_csv(path)
    # parse timestamp column
    if 'local_time' in df.columns:
        df['timestamp'] = pd.to_datetime(df['local_time'])
    elif 'date' in df.columns:
        df['timestamp'] = pd.to_datetime(df['date'])
    # select & rename demand
    df = df.rename(columns={'demand': 'demand_mwh'})
    # ensure city column exists
    if 'city' not in df.columns:
        # texas file: pivot wide to long
        df = df.melt(
            id_vars=['timestamp'],
            value_vars=[c for c in df.columns if c not in ['timestamp','date']],
            var_name='city', value_name='demand_mwh'
        )
        df['city'] = df['city'].str.lower()
    else:
        df['city'] = df['city'].str.lower()
    demand_dfs.append(df[['timestamp','city','demand_mwh']])
demand_df = pd.concat(demand_dfs, ignore_index=True)

# --- 4. Merge weather & demand ---
combined_df = pd.merge(
    weather_df,
    demand_df,
    on=['timestamp','city'],
    how='left'
)

# --- 5. Remove duplicates ---
# If multiple records share the same timestamp & city, keep the first valid demand
combined_df = combined_df.sort_values(['timestamp','city','demand_mwh'], na_position='first')
combined_df = combined_df.drop_duplicates(subset=['timestamp','city'], keep='last')

# --- 6. Final cleanup & save ---
cols = ['timestamp','city','temperature','humidity','windSpeed','demand_mwh']
avail = [c for c in cols if c in combined_df.columns]
combined_df = combined_df[avail]

# inspect
print(combined_df.info())
print(combined_df.head())

# save
combined_df.to_csv("combined_data.csv", index=False)
print(f"Saved merged dataset to {out_path}")


<class 'pandas.core.frame.DataFrame'>
Index: 165740 entries, 129400 to 260403
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   timestamp    165740 non-null  datetime64[ns]
 1   city         165740 non-null  object        
 2   temperature  165730 non-null  float64       
 3   humidity     165730 non-null  float64       
 4   windSpeed    165687 non-null  float64       
 5   demand_mwh   160895 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 8.9+ MB
None
                 timestamp       city  temperature  humidity  windSpeed  \
129400 2018-07-01 07:00:00         la        65.45      0.79       4.23   
260405 2018-07-01 07:00:00    phoenix        86.82      0.16       4.30   
162552 2018-07-01 07:00:00  san diego        61.71      0.80       3.53   
179128 2018-07-01 07:00:00   san jose        67.78      0.60       6.36   
309499 2018-07-01 07:00:00    seattle  

In [3]:
# --- 6. Count missing values per feature ---
missing_counts = combined_df.isna().sum()
print("Missing values per column:")
print(missing_counts)

Missing values per column:
timestamp         0
city              0
temperature      10
humidity         10
windSpeed        53
demand_mwh     4845
dtype: int64
